# 5.1 读写文本数据

In [12]:
with open('somefile.txt', 'rt') as f:
    data = f.read()

### rt是因为*nix和windows是用不同的字符表示换行符

# 5.2 打印输出至文件中

In [13]:
with open('test.txt', 'wt') as f:
    print('Hello World!', file=f)
    
# 文件必须是以文本模式打开，要是以二进制打开，打印则会出错

# 5.3 使用其他分隔符或行终止符打印

In [14]:
print('ACME', 50, 91.5)
print('ACME', 50, 91.5, sep=',')
print('ACME', 50, 91.5, sep=',', end='!!\n')

ACME 50 91.5
ACME,50,91.5
ACME,50,91.5!!


In [15]:
row = ('ACME', 50, 91.5)
print(*row, sep=',')

ACME,50,91.5


# 5.4 读写字节数据

In [16]:
t = 'Hello World'
t[0]

'H'

In [17]:
b = b'Hello World'
b[0]

72

In [18]:
with open('somefile.bin', 'rb') as f: 
    data = f.read(16)
    text = data.decode('utf-8')
    
with open('somefile.bin', 'wb') as f: 
    text = 'Hello World' 
    f.write(text.encode('utf-8'))

FileNotFoundError: [Errno 2] No such file or directory: 'somefile.bin'

In [ ]:
# 二进制I/O可以把数组和C结构体直接写入

import array

nums = array.array('i', [1, 2, 3, 4])
with open('data.bin', 'wb') as f:
    f.write(nums)

In [ ]:
a = array.array('i', [0, 0, 0, 0, 0, 0, 0, 0])

with open('data.bin', 'rb') as f:
    f.readinto(a)

a

# 5.5 文件不存在才能写入

In [ ]:
# 如果想要只有当文件不存在时才能写入则可以用'x'来替代'w'

with open('somefile', 'wt') as f:
    f.write('Hello\n')

with open('somefile', 'xt') as f:
    f.write('Hello\n')

# 5.6 字符串的I/O操作

In [ ]:
import io

s = io.StringIO()
s.write('Hello World\n')

In [ ]:
print('This is a test', file=s)

In [ ]:
s.getvalue()

In [ ]:
s = io.StringIO('Hello\nWor\n')

In [ ]:
s.read(4)

In [ ]:
s.read()

# 5.7 读写压缩文件

In [19]:
import gzip
import bz2

with gzip.open('somefile.gz', 'rt') as f:
    text = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'somefile.gz'

# 5.8 固定大小记录的文件迭代

In [20]:
from functools import partial

RECORD_SIZE = 32

with open('somefile.data', 'rb') as f:
    records = iter(partial(f.read, RECORD_SIZE), b'')
    for r in records:
        pass

FileNotFoundError: [Errno 2] No such file or directory: 'somefile.data'

# 5.9 读写二进制数据到可变缓冲区中

In [22]:
import os.path

def read_into_buffer(filename):
    buf = bytearray(os.path.getsize(filename))
    with open(filename, 'rb') as f:
        f.readinto(buf)
    return buf

In [23]:
with open('sample.bin', 'wb') as f:
    f.write(b'Hello World')
    
buf = read_into_buffer('sample.bin')
buf

bytearray(b'Hello World')

In [25]:
buf[0:5] = b'Hallo'

In [26]:
buf

bytearray(b'Hallo World')

In [29]:
with open('newsample.bin', 'wb') as f:
    f.write(buf)
    
with open('newsample.bin', 'rb') as f:
    f.read()

In [30]:
record_size = 32

buf = bytearray(record_size)
with open('somefile.bin', 'rb') as f:
    while True:
        n = f.readinto(buf)
        if n < record_size:
            break

FileNotFoundError: [Errno 2] No such file or directory: 'somefile.bin'

# 5.10 内存映射的二进制文件

In [35]:
import os
import mmap

def memory_map(filename, access=mmap.ACCESS_WRITE):
    size = os.path.getsize(filename)
    fd = os.open(filename, os.O_RDWR)
    return mmap.mmap(fd, size, access=access)

In [33]:
# 用于创建指定大小的文件

size = 1000000
with open('data', 'wb') as f:
    f.seek(size-1)
    f.write(b'\x00')

In [37]:
m = memory_map('data')
len(m)
m[0:11] = b'Hello World'
m.close()

with open('data', 'rb') as f:
    print(f.read(11))

b'Hello World'


In [38]:
with memory_map('data') as m:
    print(len(m))
    print(m[0:11])

1000000
b'Hello World'


In [39]:
m.closed

True

# 5.11 文件路径名的操作

In [40]:
import os

path = '/Users/beazley/Data/data.csv'

os.path.basename(path)

'data.csv'

In [41]:
os.path.dirname(path)

'/Users/beazley/Data'

In [42]:
os.path.join('tmp', 'data', os.path.basename(path))

'tmp/data/data.csv'

In [43]:
path = '~/Data/data.csv'

os.path.expanduser(path)

'/Users/JJW/Data/data.csv'

In [44]:
os.path.split(path)

('~/Data', 'data.csv')

# 5.12 测试文件是否存在

In [45]:
os.path.exists('/etc/passwd')

True

In [46]:
os.path.exists('/tmp/spam')

False

In [47]:
os.path.isfile('/etc/passwd')

True

In [48]:
os.path.isdir('/etc/passwd')

False

In [49]:
os.path.islink('/usr/local/bin/python3')

False

In [50]:
os.path.realpath('/usr/local/bin/python3')

'/usr/local/bin/python3'

In [51]:
os.path.getsize('/etc/passwd')

5925

In [52]:
os.path.getmtime('/etc/passwd')

1442372421.0

In [53]:
import time

time.ctime(os.path.getmtime('/etc/passwd'))

'Wed Sep 16 11:00:21 2015'

In [56]:
os.path.getatime('/etc/passwd')

1463815151.0

In [57]:
# 在使用os.path时有时得注意权限问题